# Purpose of the analysis
Construction of a predictive model for droughts based on weather conditions which will, in its final form, enable local water suppliers to set appropriate monthly quantities of water consumption based on its projected scarcity.
Note that for the purpose of the challenge the model was built and trained on 2015 California data.

# Load the data
The two datasets in use are:
1. United States Droughts by County built by the United States Drought Monitor: https://www.kaggle.com/us-drought-monitor/united-states-droughts-by-county   
2. California CIMIS Weather Stations daily data, downloaded and collected into a dataset by us from: http://ipm.ucanr.edu/WEATHER/wxactstnames.html        

The first contains weekly observations about the extent and severity of drought in each county of the United States. The dataset contains the following fields:
1. `releaseDate`: when this data was released on the USDM website
2. `FIPS`: the FIPS code for this county
3. `county`: the county name
4. `state`: the state the county is in
5. `NONE`: percentage of the county that is not in drought
6. `D0`: percentage of the county that is in abnormally dry conditions
7. `D1`: percentage of the county that is in moderate drought
8. `D2`: percentage of the county that is in severe drought
9. `D3`: percentage of the county that is in extreme drought
10. `D4`: percentage of the county that is in exceptional drought
11. `validStart`: the starting date of the week that these observations represent
12. `validEnd`: the ending date of the week that these observations represent
13. `domStatisticFormatID`: which is always 1    
Note: the drought categories are cumulative: if an area is in `D3`, then it is also in `D2`, `D1`, and `D0`. This means that, for every observation, `D4 <= D3 <= D2 <= D1 <= D0`.

In [1]:
import numpy as np
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
#A portion of the code has been removed as it accesses a file in the IBM Cloud Object Storage.
body = client_e2de682dc7ed4e2381d11612ff0c7b75.get_object(Bucket='waterstudio-donotdelete-pr-nm59bgkdjtmivj',Key='us-droughts.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

droughts = pd.read_csv(body)

droughts.head()

,releaseDate,FIPS,county,state,NONE,D0,D1,D2,D3,D4,validStart,validEnd,domStatisticFormatID
0,2000-11-07,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-11-07,2000-11-13,1
1,2000-10-31,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-31,2000-11-06,1
2,2000-10-24,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-24,2000-10-30,1
3,2000-10-17,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-17,2000-10-23,1
4,2000-10-10,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-10,2000-10-16,1


The second contains daily weather observations. The dataset contains the following fields:
1. `Station`: station ID
2. `Date`: date of the observation	
3. `Time`: time of the observation
4. `Precip`: precipitations' daily total measured in a 20 cm (8 in) diameter gauge
5. `type`: precipitation type
6. `Air max`: daily max air temperature measured at 1.5 m (4.92 ft)
7. `min`: daily min air temperature measured at 1.5 m (4.92 ft)
8. `obs`: not explained and not relevant for the analysis 
9. `Wx`: qualitative indication of the weather (not relevant for the analysis)
10. `Wind dir`: daily average of the wind direction measured at 2 m (6.5 ft)
11. `speed`: daily average of the wind speed measured at 2 m (6.5 ft)
12. `Bulb wet`: not explained and not relevant for the analysis
13. `dry`: not explained and not relevant for the analysis
14. `Soil max`: daily max soil temperature measured at a 15 cm (6 in) depth
15. `min.1`: daily min soil temperature measured at a 15 cm (6 in) depth
16. `Evap`: not explained and not relevant for the analysis
17. `Solar`: Daily global radiation measured by Licor pyranometer at 2 m (6.5 ft)
18. `ETo`: Evotranspiration, calculated from CIMIS hourly values
19. `RH max`: Daily max relative humidity measured at 1.5 m 
20. `min.2`:  Daily min relative humidity measured at 1.5 m 
21. `to be removed`: artifact of the data collection process containing all missing values

In [2]:
body = client_e2de682dc7ed4e2381d11612ff0c7b75.get_object(Bucket='waterstudio-donotdelete-pr-nm59bgkdjtmivj',Key='stationsData.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sensors = pd.read_csv(body)

sensors.head()

,Station,Date,Time,Precip,type,Air max,min,obs,Wx,Wind dir,...,Bulb wet,dry,Soil max,min.1,Evap,Solar,ETo,RH max,min.2,to be removed
0,ALTURAS.A,20150101,2359.0,0.0,NaN,36.0,7.0,NaN,NaN,0,...,NaN,NaN,35.0,34.0,NaN,217.0,0.03,90.0,33.0,NaN
1,ALTURAS.A,20150102,2359.0,0.0,NaN,43.0,11.0,NaN,NaN,0,...,NaN,NaN,35.0,34.0,NaN,215.0,0.03,91.0,37.0,NaN
2,ALTURAS.A,20150103,2359.0,0.0,NaN,51.0,16.0,NaN,NaN,0,...,NaN,NaN,34.0,34.0,NaN,212.0,0.04,93.0,43.0,NaN
3,ALTURAS.A,20150104,2359.0,0.0,NaN,50.0,25.0,NaN,NaN,0,...,NaN,NaN,34.0,34.0,NaN,134.0,0.03,93.0,51.0,NaN
4,ALTURAS.A,20150105,2359.0,0.0,NaN,57.0,26.0,NaN,NaN,0,...,NaN,NaN,34.0,34.0,NaN,159.0,0.03,93.0,34.0,NaN


# Data Preparation

Definition of some variables that will be needed for the subsequent transformations

In [3]:
map={'ATASCADERO.A': 'San Luis Obispo County', 'Auburn.A': 'Placer County', 'BENNETT_VALLEY.A': 'Sonoma County',
'Big_Bear_Lake.A': 'San Bernardino County','BISHOP.A': 'Inyo County','Black_Point.A': 'Marin County',
'BLYTHE_NE.A': 'Riverside County','Borrego_Springs.A': 'San Diego County',
'BRNTWOOD.A': 'Contra Costa County','BRWNSVLY.A': 'Yuba County','BUNTNGVL.A': 'Lassen County','BRAWLEY.A': 'Imperial County',
'CAMARILLO.A': 'Ventura County','CAMINO.A': 'El Dorado County','NAPA.A': 'Napa County','Coalinga.A': 'Fresno County',
'CUYAMA.A': 'Santa Barbara County','DAVIS.A': 'Yolo County','SNTACRUZ.A': 'Santa Cruz County','Delano.A': 'Tulare County',
'Denair_II.A': 'Stanislaus County','DIXON.A': 'Solano County','DURHAM.A': 'Butte County','FAIR_OAKS.A': 'Sacramento County',
'Gerber_South.A': 'Tehama County','Gilroy.A': 'Santa Clara County','GLENDALE.A': 'Los Angeles County','SNTA_ANA.A': 'Orange County',
'KESTERSN.A': 'Merced County','MANTECA.A': 'San Joaquin County','MCARTHUR.A': 'Shasta County','TULELK2.A': 'Siskiyou County',
'HOPLAND2.A': 'Mendocino County','HOLLISTR.A': 'San Benito County','UNION_CITY.A': 'Alameda County','Plymouth_II.A': 'Amador County',
'Verona.A': 'Sutter County','ALTURAS.A': 'Modoc County','ARYOSECO.A': 'Monterey County','ARVIN.A': 'Kern County'}

timemap={'01':0, '02': 31, '03':49, '04':90, '05':120, '06':151, '07':181, '08':212, '09':243, '10': 273, '11':304, '12':334}

list1=['Precip','Air max', 'min', 'speed','Soil max','min.1','Solar','ETo', 'RH max', 'min.2']

map2={'Alpine County': 0,'Calaveras County': 0,'Colusa County':0,'Glenn County':0,'Kings County':0,'Lake County':0,'Madera County':0,
'Mariposa County':0,'Mono County':0,'Nevada County':0,'Plumas County':0,'Sierra County':0,'Tuolumne County':0,'Del Norte County':0,
'Humboldt County':0,'San Francisco County':0,'San Mateo County':0,'Trinity County':0}

Elimination of the variables containing mostly missing values, fill in of the remaining ones using the mean of the respective variables

In [4]:
sensors.drop(['to be removed','type','obs','Wx','Bulb wet','dry','Evap','Wind dir'],axis=1, inplace=True)
sensors.fillna(sensors.mean(),inplace=True)


Transformation of the stations into their respective counties

In [5]:
sensors.replace({'Station': map}, inplace=True)
sensors.rename(columns = {'Station':'County'}, inplace = True)

Construction of the `Year` and `Week` variable both based on the Date one 

In [6]:
sensors.insert(13,"Year", None)
sensors.insert(14,"Week", None)

for index, row in sensors.iterrows():
    temp=row['Date']
    year=str(temp)[0:4]
    month=str(temp)[4:6]
    day=str(temp)[6:8]
    sensors.at[index,'Year']=year
    dayselapsed=3+timemap[month]+int(day)
    weeks=np.ceil(dayselapsed/7)
    sensors.at[index,'Week']=weeks
sensors.drop(['Date','Time'],axis=1, inplace=True)

Sorting of the dataset and filtering out of all the data which covers the beginning of 2016

In [7]:
sensors.sort_values(['Week','County'])
sensors=sensors[sensors.Year!='2016']

Visualization of the final `sensors` dataset

In [8]:
sensors.head()

,County,Precip,Air max,min,speed,Soil max,min.1,Solar,ETo,RH max,min.2,Year,Week
0,Modoc County,0.0,36.0,7.0,0.0,35.0,34.0,217.0,0.03,90.0,33.0,2015,1
1,Modoc County,0.0,43.0,11.0,0.0,35.0,34.0,215.0,0.03,91.0,37.0,2015,1
2,Modoc County,0.0,51.0,16.0,0.0,34.0,34.0,212.0,0.04,93.0,43.0,2015,1
3,Modoc County,0.0,50.0,25.0,0.0,34.0,34.0,134.0,0.03,93.0,51.0,2015,1
4,Modoc County,0.0,57.0,26.0,0.0,34.0,34.0,159.0,0.03,93.0,34.0,2015,2


Construction of the `sensors_means` dataset made of weekly averages of the information from the weather stations. Note that this is done in order to be consistent with the `droughts` dataset weekly structure

In [9]:
sensors_means=pd.DataFrame(columns=['county','Precip','Air max', 'min', 'speed','Soil max','min.1','Solar','ETo', 'RH max', 'min.2', 'Week'])

weekindex=0
countyindex=0
i=0

for i in range(0,14600):
    if i!=0 and sensors.iloc[i]['County']!=sensors.iloc[i-1]['County']:
        weekindex=0
        countyindex+=1
    if sensors.iloc[i]['Week']>weekindex:
        ma=i
        while True:
            ma+=1
            if ma==14600:
                break
            if sensors.iloc[ma]['Week']!=sensors.iloc[ma-1]['Week']:
                break
        delta=ma-i
        iforlater=i
        index=countyindex*53+weekindex
        sensors_means.at[index,list1]=0
        sensors_means.at[index,list1]=sensors_means.loc[index][list1]+np.sum(sensors.iloc[i:ma][list1])
        if not delta==0:
            sensors_means.at[index,list1]=sensors_means.loc[index][list1]/delta
        sensors_means.at[index,'county']=sensors.iloc[iforlater]['County']
        weekindex+=1
        sensors_means.at[index,'Week']=weekindex 

Visualization of the final `sensors_means` dataset

In [10]:
sensors_means.head()

,county,Precip,Air max,min,speed,Soil max,min.1,Solar,ETo,RH max,min.2,Week
0,Modoc County,0,45,14.75,0,34.5,34,194.5,0.0325,91.75,41,1
1,Modoc County,0.00285714,55.2857,24.8571,0,34.5714,34.1429,176.714,0.0342857,94.5714,47.4286,2
2,Modoc County,0.0257143,47.1429,27.7143,0,38.2857,36.7143,115.143,0.03,94.5714,58.8571,3
3,Modoc County,0,52.7143,22.2857,0,38.5714,36.8571,231.143,0.0471429,93.8571,37.7143,4
4,Modoc County,0.00285714,56,23.4286,0,39.4286,37.4286,239.857,0.0557143,93.5714,37.2857,5


Removal of all the datapoints where no drought was found, removal of the `releaseDate` column, and selection of the data related to California

In [11]:
droughts = droughts[droughts.NONE != 100]
droughts.drop(['releaseDate'], axis=1, inplace=True)
droughts = droughts[droughts.state == 'CA']

Removal of the datapoints for which no CIMIS Weather Station data is available

In [12]:
droughts.replace({'county': map2}, inplace=True)
droughts = droughts[droughts.county != 0]

Creation of the `Week` column staring from the `validStart` and `validEnd` of the observation and of the `D_score` column which we will use to build our target variable

In [13]:
droughts.insert(12,"Week", None)
droughts.insert(13,"D_score", None)

for index, row in droughts.iterrows():
    temp=row['validStart']
    year=temp[0:4]
    month=temp[5:7]
    day=temp[8:10]
    if (int(year)>2015) or int(year)<2015:
        droughts.drop(index,axis=0, inplace=True)
    else:
        dayselapsed=3+timemap[month]+int(day)
        weeks=np.ceil(dayselapsed/7)
        droughts.at[index,'Week']=weeks
        droughts.at[index,'D_score']=np.sum(row[['D0','D1','D2','D3','D4']])
droughts.drop(['validStart','validEnd','FIPS','state','domStatisticFormatID','D0','D1','D2','D3','D4','NONE'], axis=1, inplace=True)

Visualization of the final `droughts` dataset

In [14]:
droughts.head()

,county,Week,D_score
2504293,Alameda County,45,418.96
2504294,Alameda County,44,418.96
2504295,Alameda County,43,418.96
2504296,Alameda County,42,418.96
2504297,Alameda County,41,418.96


Construction of the `final` dataset combining the `droughts` and `sensors_means` ones and removal of the variables used for the join

In [15]:
final=pd.merge(droughts, sensors_means, on=['county','Week'])
final.drop(['county','Week'],axis=1, inplace=True)

Construction of the our target variable: a 3 categories variable indicating the level of severity of the droughts

In [16]:
for index, row in final.iterrows():
    if int(row['D_score'])>490:
        final.at[index, 'D_score']='Severe'
    elif int(row['D_score'])<395:
        final.at[index, 'D_score']='Moderate'
    else:
        final.at[index, 'D_score']='Intense'

# Fitting models and evaluating performances

Split between training and test data. Given the dataset size the `test` set is chosen to be 20% of the datapoints

In [17]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(final, test_size=0.20)

Logistic regression fitting, estimation, and evaluation (through the `accuracy_score` and a `confusion_matrix`)

In [19]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=1000000000000,multi_class='ovr')
logreg.fit(train.drop(['D_score'],axis=1),train.loc[:,'D_score'])
results=logreg.predict(test.drop(['D_score'],axis=1))

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(test.loc[:,'D_score'],results)

0.5144230769230769

In [21]:
confusion_matrix(test.loc[:,'D_score'],results)

array([[85, 17, 35],
       [42, 53, 35],
       [55, 18, 76]])

Random forest fitting, estimation, and evaluation (through the `accuracy_score` and a `confusion_matrix`)

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(train.drop(['D_score'],axis=1),train.loc[:,'D_score'])
results=rf.predict(test.drop(['D_score'],axis=1))

In [23]:
accuracy_score(test.loc[:,'D_score'],results)

0.625

In [24]:
confusion_matrix(test.loc[:,'D_score'],results)

array([[ 79,  27,  31],
       [ 22,  74,  34],
       [ 26,  16, 107]])